<a href="https://colab.research.google.com/github/ambicapvs/ambica_info5731_spring2021/blob/main/Ambica_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [91]:
#Collecting Data from IMDB 

!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
from selenium import webdriver
import csv
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver =webdriver.Chrome('chromedriver',options=chrome_options) #setup chromedriver options to run in googlecolab 
driver.get('https://www.imdb.com/title/tt7286456/reviews')
for i in range(5):    #each page has approx. 24 reviews. 
        try:
            driver.implicitly_wait(5)
            driver.find_element_by_id("load-more-trigger").click()
        except Exception as e:
          print(e) 

reviews = driver.find_elements_by_class_name("show-more__control") #extract reviews and store in csv file
print("saving data into csv file")
with open('new_csv.csv','w') as csv_file:
  csvwriter = csv.DictWriter(csv_file, fieldnames=["ReviewID", "Review"])
  csvwriter.writeheader()
  i=1
  for review in reviews:
    if(review.text!= ""):
      csvwriter.writerow({"ReviewID": str(i), "Review": review.text})
      i=i+1
driver.close()

#Cleaning

import pandas as pd
dataframe = pd.read_csv('/content/new_csv.csv',sep=',',engine ='python', header=0)
dataframe.keys()

#remove punctuation
dataframe["punctuationremoval"] = dataframe["Review"].str.replace('[^\w\s]','')
#remove special chars
dataframe['special_chars'] = dataframe["punctuationremoval"].str.replace('[^A-Za-z0-9 ]+','')
#remove Numbers
dataframe['nums_removed'] = dataframe["special_chars"].str.replace('[^A-Za-z ]+','')

#extract stopwords
import requests
data = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
a = data.text
stop = a.split()

#stopwords
dataframe['stopwords'] = dataframe["nums_removed"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

#lowercase
dataframe["lower_case"] = dataframe['stopwords'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#tokenizing data for next step - stemming
import nltk
from textblob import TextBlob
nltk.download('punkt')
dataframe["tokenization"] = dataframe["lower_case"].apply(lambda x: TextBlob(x).words)
dataframe

#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
dataframe["stemming"]= dataframe["tokenization"].apply(lambda x: " ".join([st.stem(word) for word in x]))

#Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')
dataframe["lemmatization"] = dataframe["stemming"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
dataframe

#save cleaned data to csv file
dataframe.to_csv("resultfile.csv")



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [92]:
#(1) Count the frequency of all the N-grams (N=3).
from nltk.util import ngrams
listToStr = ' '.join([str(item) for item in dataframe["lemmatization"]]) 
trigrams = ngrams(listToStr.split(), 3) # N=3
freq_tri = nltk.FreqDist(trigrams)
freq_tri


FreqDist({('everi', 'movi', 'come'): 1,
          ('movi', 'come', 'truli'): 1,
          ('come', 'truli', 'make'): 1,
          ('truli', 'make', 'impact'): 1,
          ('make', 'impact', 'joaquin'): 1,
          ('impact', 'joaquin', 'perform'): 1,
          ('joaquin', 'perform', 'scenographi'): 1,
          ('perform', 'scenographi', 'brillianc'): 1,
          ('scenographi', 'brillianc', 'grotesqu'): 1,
          ('brillianc', 'grotesqu', 'haunt'): 1,
          ('grotesqu', 'haunt', 'cringi'): 1,
          ('haunt', 'cringi', 'hard'): 1,
          ('cringi', 'hard', 'watch'): 1,
          ('hard', 'watch', 'time'): 1,
          ('watch', 'time', 'mesmer'): 1,
          ('time', 'mesmer', 'wont'): 1,
          ('mesmer', 'wont', 'blink'): 1,
          ('wont', 'blink', 'eye'): 1,
          ('blink', 'eye', 'watch'): 1,
          ('eye', 'watch', 'tragic'): 1,
          ('watch', 'tragic', 'serious'): 1,
          ('tragic', 'serious', 'funni'): 1,
          ('serious', 'funni', '

In [93]:
#(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.
#Probabilities of bigrams

from collections import Counter
bigrams = ngrams(listToStr.split(), 2)
freq_bi = nltk.FreqDist(bigrams)
bigramdict = dict(freq_bi)
for bigram in bigramdict:
  print( bigram , str(bigramdict[bigram] / listToStr.count(bigram[0])))


Streaming output truncated to the last 5000 lines.
('lazi', 'write') 1.0
('write', 'focus') 0.09090909090909091
('focus', 'fanci') 0.3333333333333333
('fanci', 'long') 1.0
('long', 'shot') 0.06666666666666667
('shot', 'joaquin') 0.5
('phoenix', 'danc') 0.016666666666666666
('danc', 'crazili') 0.16666666666666666
('crazili', 'produc') 1.0
('produc', 'someth') 0.16666666666666666
('someth', 'valu') 0.08333333333333333
('valu', 'franchis') 0.3333333333333333
('franchis', 'a') 1.0
('a', 'beyond') 0.0003676470588235294
('beyond', 'pathet') 0.3333333333333333
('pathet', 'taxi') 0.5
('taxi', 'driverk') 0.06666666666666667
('driverk', 'comedi') 1.0
('comedi', 'ripoff') 0.16666666666666666
('ripoff', 'borefest') 1.0
('borefest', 'object') 1.0
('object', 'singl') 1.0
('singl', 'worstcomplet') 0.2
('worstcomplet', 'nonexist') 1.0
('nonexist', 'version') 1.0
('version', 'joker') 0.36363636363636365
('joker', 'charact') 0.039473684210526314
('charact', 'historyth') 0.016129032258064516
('historyth'

In [94]:
#(3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).

from textblob import TextBlob
import nltk
nltk.download('brown')
nounvalues = []
freqvalues =[]

#Generate a list of Noun Phrases
for review in dataframe["lemmatization"]:
  blob = TextBlob(review)
  for i in blob.noun_phrases:
    nounvalues.append(i)

#Generate a list of Noun Frequencies
for item in nounvalues:
  nounfreq = []
  for review in dataframe["lemmatization"]:
    nounfreq.append(review.count(item) / listToStr.count(item))
  freqvalues.append(nounfreq)

nDataFrame = pd.DataFrame(freqvalues).T
nDataFrame.columns = list(nounvalues)

lens = len(dataframe["lemmatization"])
indexvalues = ["Review #"+str(item) for item in range(1, lens+1)]
nDataFrame.index = indexvalues
nDataFrame


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


,everi movi,impact joaquin,scenographi brillianc grotesqu haunt cringi,time mesmer wont blink eye,serious funni moment emot rollercoast sometim multipl emot poppingup timethi,typic actionriddl,superhero movi,proper psycholog thrillerdrama singl,thi movi,alon isol truli relat,sorri charact,lot peopl,encourag violenc,truli movi encourag everi,u becom,everyon respect,truli masterpiec,hollywood film,film decad,comic book chillingli realist,real ife remark,direct cinematographi music act,peopl surpris,violent necess messag,societi reflect underappreciatedunrecognizedbulli peopl,way show class differ,corrupt rich talent rule,multipl joker,bitter way film show,peopl uncomfort peopl consid wake,perfect film,joaquin phoenix,fearless stun emot depth physic,referenc heath ledger oscarwin,dark knight,wide consid definit liveact portray joker,fact everyon,stun phoenix,imposs portray,potenti exce,...,movi i hadnt,trailer anyth decid,stone mate steve,ticket differ movi couldnt,screen end screen show,wasnt til minut i realis movi good i couldnt,certain actor,realli mumbl,low qualiti,stori confus,lot scene liter noth,peopl talk walk corridor i whisper mate steve hour,asleepth good thing movi act geezer play joker good,funni scene,end movi overli,long complex,direct produc poorli opportun,joaquin phoenix blowngratuit violenc misplac violenc,wrong approach jokerdisappoint,viewer film wander,long period ton footag,room floori realli,stirringli creepi,joaquin joker,wasnt thereif look joaquin moment,gladiat disappoint,wow movi i,joaquin phoenix,joker i surpris i wasnt,pull i hope wasnt,anoth cheesi rendit,full fluff unrealist explan joker,adult son wouldnt,nudg wake i pleasantli surpris,todd phillip version play joaquin act superb,anguish charact,act movi,deriv dc,thi movi poorli,dark movi show problem
Review #1,1.0,1.0,1.0,1.0,1.0,1.0,0.125,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review #2,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.038462,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review #3,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review #4,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03125,1.0,1.0,0.2,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review #5,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Review #102,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [95]:
# (1) To build the documents-terms weights (tf*idf) matrix bold text.

import math
import re
def calc_tfidf(review,word):
  wordcount = len(re.findall(r'\w+', review)) 
  tf = review.count(word)/wordcount
  if(tf!=0):
    idf = math.log(wordcount)/review.count(word)
    return(tf*idf)
  else:
    return(0)
  
terms = set([i for j in dataframe["lemmatization"] for i in j.split()])
tf_idf = pd.DataFrame(terms,columns=["terms"])
count = 0

#TF-iDF Matrix
for review in dataframe["lemmatization"]:
  tf_idf[str(count)] = tf_idf["terms"].apply(lambda x : calc_tfidf(review,x))
  count+=1
tf_idf





,terms,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
0,jokerjust,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0305,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,season,0.0,0.0,0.000000,0.0,0.0,0.050879,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,franchis,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.04957,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,millenni,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.027154,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
4,evil,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.055309,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.056983,0.000000,0.046785,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0305,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,fluff,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.062757,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.060693,0.0
2006,hollywood,0.0,0.0,0.052266,0.0,0.0,0.000000,0.0,0.033404,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.038873,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.045694,0.0

In [96]:
# (2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using cosine similarity.

myquery = "Every once in a while a movie comes, that truly makes an impact. Joaquin's performance and scenography in all it's brilliance. Grotesque, haunting and cringy. Hard to watch at times,... but so mesmerizing, you won't blink an eye watching it. Tragic, but with seriously funny moments. Emotional rollercoaster - sometimes, with multiple emotions popping-up at the same time. this is far from a typical action-riddled predictable super-hero movie - it's a proper psychological thriller/drama, with the single best character development I have ever seen."

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('stopwords')

#Cleaning the myquery
tokenizedquery = word_tokenize(myquery)
stops = stopwords.words('english') 
tokens_without_sw = [word for word in tokenizedquery if not word in stops]
tokens_lowercase = map(lambda x:x.lower(),tokens_without_sw)
lowercased = list(tokens_lowercase)
numsremoved = [x for x in lowercased if not x.isnumeric()]
puncremoved = [''.join(x for x in par if x not in string.punctuation) for par in numsremoved]
str_list = list(filter(None, puncremoved))
X_set = set(str_list)

#Calculate Cosine Similarity reference - https://www.geeksforgeeks.org/python-measure-similarity-between-two-sentences-using-cosine-similarity/

def calc_similarity(X_set, Y_set):
  rvector = X_set.union(Y_set)
  l1 =[]
  l2 =[]
  c= 0 
  for w in rvector: 
      if w in X_set: 
        l1.append(1)
      else: 
        l1.append(0) 
      if w in Y_set: 
        l2.append(1) 
      else: 
        l2.append(0)
  for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
  cosine = c / float((sum(l1)*sum(l2))**0.5) 
  return cosine

sim_column = []

#Compare Dataframe of collectected reviews. 
for review in dataframe["lemmatization"]:
  df_tokenized = word_tokenize(review)
  Y_set = set(df_tokenized)
  similarity = calc_similarity(X_set, Y_set)
  sim_column.append(similarity)
cDataframe = pd.DataFrame(list(zip(dataframe['Review'],sim_column)), columns=['Review','Similarity'])
cDataframe.index = indexvalues
cDataframe






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Review,Similarity
Review #1,"Every once in a while a movie comes, that trul...",0.362030
Review #2,This is a movie that only those who have felt ...,0.000000
Review #3,"Truly a masterpiece, The Best Hollywood film o...",0.017857
Review #4,Joaquin Phoenix gives a tour de force performa...,0.022043
Review #5,Most of the time movies are anticipated like t...,0.083351
...,...,...
Review #102,"One of the worst movies I have ever seen. Ok, ...",0.089363
Review #103,I didnt really know much about this movie. I h...,0.048526
Review #104,Film was directed and produced poorly. Opportu...,0.022588
Review #105,"Wow, what a movie! I have to admit, When I fir...",0.038526


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [97]:
# The GitHub link of your final csv file

# Link: https://github.com/ambicapvs/ambica_info5731_spring2021/blob/main/Q3(assig-3).csv